In [1]:
from IPython.core.display import HTML
import logging, json
import cobra
import biosapi

In [2]:
%run ../../scripts/bios_utils.py

In [3]:
api = biosapi.BIOS('http://192.168.1.21:8080/biosynth-web-biobase')

In [5]:
api.set_annotation_model_compartment('MitoCore', 3781218, 'MITOCHONDRIA')

<Response [200]>

In [1]:
#cmps = api.get_model_compartments('MitoCore')
#cmps

In [11]:
api.get_model_species_annotation('iJW145')['3039335']['annotation']['3855778']

{'entry': 'META:THR',
 'database': 'MetaCyc',
 'link': {'updated_at': 1566661494766,
  'created_at': 1529950300082,
  'authors': 'system:1'}}

In [12]:
api.delete_annotation_model_species('iJW145', 3039335, 'MetaCyc', 'META:THR')

<Response [200]>

In [4]:
model_id = 'iWV1213'

In [5]:
model_cmps = api.get_model_compartments(model_id)
model_spis = api.get_model_species(model_id)
model_rxns = api.get_model_reactions(model_id)
model_genes = api.get_model_genes(model_id)

In [6]:
b = biosapi.io.BiosModelToCobraBuilder(model_cmps, model_spis, model_rxns, model_genes)
cobra_model = b.build()
cobra_model

set()


Name,model
Memory address,0x07fb928ffd890
Number of metabolites,1413
Number of reactions,1326
Number of groups,0
Objective expression,0
Compartments,"Peroxisome, Cytoplasm, Mitochondrion, Periplasm, Extracellular"


In [7]:
for o in model_spis:
    b.convert_modelcompound(o)

In [10]:
o

{'major_label': 'MetaboliteSpecie',
 'bios_references': [],
 'notes': '<notes>\n          <body xmlns="http://www.w3.org/1999/xhtml">\n            <p>FORMULA: </p>\n            <p>CHARGE: 0</p>\n          </body>\n        </notes>',
 'constant': 'false',
 'bios_spi_degree': 0,
 'hasOnlySubstanceUnits': 'false',
 'created_at': 1545980681050,
 'bios_id': 3786546,
 'entry': 'species_3981_7@iWV1213',
 'proxy': False,
 'boundaryCondition': 'false',
 'updated_at': 1545980681050,
 'compartment': 'c',
 'name': 'H2O[c]+[c]'}

In [8]:
cobra_model_sbml = cobra.io.read_sbml_model('/Users/fliu/workspace/jupyter/data/sbml/iMM904/source.xml')

In [9]:
cobra_model_sbml

Name,iMM904
Memory address,0x07fa598325cd0
Number of metabolites,1226
Number of reactions,1577
Number of groups,59
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, mitochondria, extracellular space, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, nucleus, golgi apparatus"


In [31]:
for m in cobra_model.metabolites:
    m_other = cobra_model_sbml.metabolites.get_by_id(m.id[2:])
    assert(m_other.compartment == m.compartment)
    assert(m_other.name == m.name)
    assert(len(m_other.reactions) == len(m.reactions))

In [33]:
assert(cobra_model.compartments == cobra_model_sbml.compartments)

In [49]:
for r in cobra_model.reactions:
    r_other = cobra_model_sbml.reactions.get_by_id(r.id[2:])
    #print(r_other.name, r.name)
    try:
        #assert(r_other.name == r.name)
        assert(r_other.compartments == r.compartments)
    except AssertionError:
        print(r.id, r_other.name)
        print(r.id, r.name)
        pass

In [50]:
for o in b.model_rxns:
    o

In [53]:
set(o.keys())

{'annotation',
 'bios_id',
 'bios_rxn_gpr_data',
 'bios_rxn_gpr_id',
 'bios_stoichiometry',
 'created_at',
 'entry',
 'fast',
 'id',
 'lowerFluxBound',
 'major_label',
 'metaid',
 'name',
 'proxy',
 'reversible',
 'sboTerm',
 'updated_at',
 'upperFluxBound'}

In [95]:
request_url = '{}/api/neo4j/data/{}'.format(api.base_url, 3780473)
resp = requests.get(request_url, headers=api.headers)
HTML(resp.content.decode('utf-8'))

In [115]:
for rxn in model_rxns:
    sbo = get_sbo_terms(rxn['bios_id'])
    name = None
    if 'name' in rxn:
        name = rxn['name']
    print(sbo, rxn['bios_id'], name, rxn['id'])
    break

None 3154577 Glycoaldehydye reversible transport R_GCALDt


In [109]:
def get_sbo_terms(node_id):
    data = api.get_bios_node_edges(node_id)
    sboterms = get_sbo_term(data)
    terms = set()
    for id in sboterms:
        sboterm_id = sboterms[id]['bios_other_node_id']
        #print(id, edge_pros)
        sboterm = api.get_bios_node(sboterm_id)
        print(sboterm)
        terms.add(sboterm['entry'])

In [111]:
#make this generic to any edge type
def get_sbo_term(edge_data):
    if 'OUTGOING' in edge_data and 'has_sbo_term' in edge_data['OUTGOING']:
        return edge_data['OUTGOING']['has_sbo_term']
    return {}

In [ ]:
request_url = '{}/api/model/{}'.format(api.base_url, model_id)
resp = requests.get(request_url, headers=api.headers)

In [13]:
request_url = '{}/api/model/{}'.format(api.base_url, model_id)
resp = requests.get(request_url, headers=api.headers)

{'R_BIOMASS_SC5_notrace'}


Name,model
Memory address,0x013267755f8
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,1.0*R_BIOMASS_SC5_notrace - 1.0*R_BIOMASS_SC5_notrace_reverse_353ca
Compartments,"c, r, m, e, x, n, v, g"


In [41]:
fmodels = [
    'iNL895', #
    'iCT646', #
    'iMM904', #
    'iTO977', #
    'iSS884', #
    'iLC915', #    
    'iAL1006', #
    'iRL766', #
    'iMA871', #
    'iJDZ836', #
    'iWV1314', #
    
    'iOD907', ##
    'iWV1213',
    'iJL1454',
    'iNX804',
    'yeast_6.06',
    'yeast_7.6',
]
for m in models:
    if m['entry'] in fmodels:
        print(m['entry'])

iOD907
iNL895
iWV1213
iRL766
iMM904
iLC915
iJDZ836
iMA871
iSS884
iAL1006
iTO977
iCT646


In [8]:
request_url = '{}/api/model'.format(api.base_url, model_id)
resp = requests.get(request_url, headers=api.headers)
models = resp.json()

In [26]:
#models

In [22]:
#data = resp.json()

In [15]:
data.keys()

dict_keys(['id', 'entry', 'name', 'source', 'version', 'description', 'md5', 'reactions', 'metabolites', 'species', 'subcellularCompartments', 'subsystems', 'genes', 'properties'])

In [162]:
logger = logging.getLogger(__name__)

In [21]:
cobra_model.summary()

IN FLUXES            OUT FLUXES            OBJECTIVES
-------------------  --------------------  -------------------
M_4abz_e    10       M_asn__L_e    10      R_BIOMASS_SC...  10
M_dttp_e    10       M_cys__L_e    10
M_gam6p_e   10       M_gal_e       10
M_hxan_e    10       M_gln__L_e    10
M_iamoh_e   10       M_pi_e        10
M_tre_e     10       M_thymd_e     10
M_tyr__L_e  10       M_trp__L_e    10
M_xtsn_e    10       M_pap_e        9.8
M_ocdca_e    9.98    M_ocdcya_e     9.48
M_ptrc_e     8.17    M_acald_e      8.09
M_ins_e      7.99    M_iamac_e      7.04
M_mal__L_e   7.78    M_pyr_e        5.22
M_chol_e     6.98    M_g3pc_e       4.75
M_nmn_e      6.33    M_Nbfortyr_e   4.49
M_lac__L_e   5.12    M_ibutoh_e     4.01
M_gthox_e    5       M_nadp_e       3.33
M_2mppal_e   4.65    M_asp__L_e     3.26
M_uri_e      4.18    M_dad_2_e      3.17
M_melib_e    3.5     M_id3acald_e   2.93
M_dcyt_e     3.44    M_xylt_e       1.87
M_pro__L_e   3.25    M_dgsn_e       1.73
M_so3_e      3.02  